In [ ]:
"""
After bitgram.ipynb, the character does not talk to each other.

Note: the current character only depends on previous history. 
Not previous history and future character.

The easiest way to talk to previous characters is to sum (or average) the previous tokens
"""